In [19]:
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    ConstantSpectralModel,
    SkyModel,
    DiskSpatialModel,
    GeneralizedGaussianSpatialModel,
    GaussianSpatialModel,
    Models,
)
from gammapy.datasets import MapDataset

from modules import write_json, read_json, plot_results
from pathlib import Path

from os import chdir
import glob
import shutil
import json
import time
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Spatial and spectral models
spectra = 1
spatial = 1

if spectra == 0:
    spectral_model = ConstantSpectralModel(const="1e1 / (cm2 s TeV)")
else:
    spectral_model = PowerLawSpectralModel(
        index=2,
        amplitude="1e-12 TeV-1 cm-2 s-1",
        reference=1 * u.TeV,
    )

if spatial == 0:
    spatial_model = GaussianSpatialModel(
        lon_0=0 * u.deg,
        lat_0=1.5 * u.deg,
        sigma="1 deg",
        e=0.01,
        phi=0 * u.deg,
        frame="galactic",
    )
elif spatial == 1:
    spatial_model = DiskSpatialModel(
        lon_0=0 * u.deg,
        lat_0=1.5 * u.deg,
        r_0="1 deg",
        e=0.01,
        phi=0 * u.deg,
        edge_width=0.1,
        frame="galactic",
    )
else:
    spatial_model = GeneralizedGaussianSpatialModel(
        lon_0=0 * u.deg,
        lat_0=1.5 * u.deg,
        eta=.5,
        r_0=12 * u.deg,
        e=.999,
        phi=90 * u.deg,
        frame="galactic",
    )


In [ ]:
# Create JSON file
with open(f"results_sliding_window.json", "w") as outfile:
    json.dump({}, outfile)
    
cwd = Path.cwd()
path = cwd / 'datasets' / 'analysis_1'

# Here you can change the "steps" of the sliding window
for i in np.arange(68, 79, 0.5):
    
    analysis = MapDataset.read(filename= path / f"{i}-dataset.fits.gz")

    chdir("..")

    filename = "skymodel/global_skymodel_v1.0.yaml"
    models_read = Models.read(filename)
    models_read.remove("diffuse")
    models_read.freeze("spatial")

    model_config = SkyModel(spectral_model=spectral_model,
                            spatial_model=spatial_model, name="cygnus_background")

    models_read.append(model_config)
    
    chdir(cwd)

    analysis.set_models(models_read, extend=False)
    analysis.models[0].parameters["lon_0"].frozen = True
    analysis.models[0].parameters["lon_0"].value = i
    analysis.models[0].parameters["lat_0"].max = 5
    analysis.models[0].parameters["lat_0"].min = -4
    if spectra == 1:
        analysis.models[0].parameters["amplitude"].max = 1.e-11
        analysis.models[0].parameters["amplitude"].min = 1.e-14
    if spatial == 1:
        analysis.models[0].parameters["r_0"].max = 5
        analysis.models[0].parameters["r_0"].min = 0
        analysis.models[0].parameters["e"].frozen = False
        analysis.models[0].parameters["phi"].frozen = True
        analysis.models[0].parameters["edge_width"].frozen = True
    if spatial != 0 and spatial != 1:
        analysis.models[0].parameters["r_0"].frozen = True
    analysis.models[1].parameters["norm"].frozen = False
    analysis.models[1].parameters["norm"].value = 1

    analysis.run_fit()

    results = {
        str(analysis.fit_result.parameters["lon_0"].value): {
            "models": [analysis.models[0].spectral_model.tag[0], analysis.models[0].spatial_model.tag[0]],
            "result": [analysis.fit_result.optimize_result.success,
                       analysis.fit_result.optimize_result.message,
                       analysis.fit_result.optimize_result.total_stat],
            analysis.models[0].spectral_model.parameters[spectra].name: analysis.models[0].spectral_model.parameters[spectra].value,
        },
    }
    for i, ele in enumerate(analysis.models[0].spatial_model.parameters.names):
        if ele == "lon_0":
            continue
        results[str(analysis.fit_result.parameters["lon_0"].value)] |= {
            ele: analysis.models[0].spatial_model.parameters[i].value
        }

    write_json(results, "results_sliding_window.json")

In [ ]:
timestr = time.strftime("%Y%m%d-%H%M%S")
(Path(timestr)).mkdir(parents = True)

files = glob.glob(f"*_sliding_window*")
for file in files:
    # extract file name form file path
    file_name = Path(file).name
    shutil.move(file, f"{timestr}/" + file_name)
    print('Moved:', file)

In [ ]:
lists = read_json(f"{timestr}/results_sliding_window.json")
plot_results(lists, spectra, spatial)